In [12]:
import h5py
import numpy as np
import tensorflow as tf
with h5py.File("data/full_dataset_vectors.h5", 'r') as h5:
    X_train, y_train = h5["X_train"][:], h5["y_train"][:]
    X_test, y_test = h5["X_test"][:], h5["y_test"][:]
print(y_train.dtype)
y_train = np.eye(10,dtype=np.int64)[y_train]
y_test = np.eye(10,dtype=np.int64)[y_test]
print(type(y_train))

int64
<class 'numpy.ndarray'>


In [5]:
#定义每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = X_train.shape[0] // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
#初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name=name)

#初始化偏置值
def bias_variable(shape,name):
    initial = tf.zeros(shape)+0.1
    return tf.Variable(initial,name=name)

#卷积层
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

#池化层
def maxpool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('input'):
    #定义两个placeholder
    x=tf.placeholder(tf.float32,[None,4096],name="x-input")
    y=tf.placeholder(tf.float32,[None,10],name="y-input")
    with tf.name_scope('keep_prob'): 
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('x_image'):
        x_image = tf.reshape(x,[-1,16,16,16],name='x_image')

with tf.name_scope('conv1'):
    with tf.name_scope('w_conv1'):
        #对第一卷积层的filter和bias做初始化
        w_conv1 = weight_variable([3,3,16,32],name='w_conv1')
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32],name='b_conv1')

    with tf.name_scope('conv2d_1'):
        #把x_image和filter做卷积，在加上bias，作用于relu激活函数
        conv2d_1 = conv2d(x_image,w_conv1)+b_conv1
    with tf.name_scope('h_conv1'):
        h_conv1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'):
        h_pool1 = maxpool(h_conv1)
        
with tf.name_scope('conv2'):
    with tf.name_scope('w_conv2'):
        #对第二卷积层的filter和bias做初始化
        w_conv2 = weight_variable([3,3,32,64],name='w_conv2')
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64],name='b_conv2')

    with tf.name_scope('conv2d_2'):
        #把第一层输出和filter做卷积，在加上bias，作用于relu函数
        conv2d_2 = conv2d(h_pool1,w_conv2)+b_conv2
    with tf.name_scope('h_conv2'):
        h_conv2 = tf.nn.relu(conv2d_2)
    with tf.name_scope('h_pool2'):
        h_pool2 = maxpool(h_conv2)

with tf.name_scope('fc1'):
    with tf.name_scope('h_pool2_flat'):
        #把池化层2的输出扁平化为1维
        h_pool2_flat = tf.reshape(h_pool2,[-1,4*4*64],name='h_pool2_flat')
    with tf.name_scope('w_fc1'):
        #初始化第一个全连接层的权值和bias
        w_fc1 = weight_variable([4*4*64,1024],name='w_fc1')
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024],name='b_fc1')

    with tf.name_scope('wx_plus_b1'):
        #求第一个全连接层的输出
        wx_plus_b1 = tf.matmul(h_pool2_flat,w_fc1)+b_fc1
    with tf.name_scope('h_fc1'):
        h_fc1 = tf.nn.relu(wx_plus_b1)
    with tf.name_scope('h_fc1_drop'):
        h_fcl_drop = tf.nn.dropout(h_fc1,keep_prob,name='h_fc1_drop')

with tf.name_scope('fc2'):
    with tf.name_scope('w_fc2'):
        #初始化第二个全连接层的权值和bias
        w_fc2 = weight_variable([1024,10],name='w_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([10],name='b_fc2')

    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2 = tf.matmul(h_fc1,w_fc2)+b_fc2
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(wx_plus_b2)

with tf.name_scope('loss'):
    #交叉熵代价函数 loss指标表示模型的好壞
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction),name='loss')
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
    with tf.name_scope('accuracy'):
        #计算准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter('logs/train',sess.graph)
    test_writer = tf.summary.FileWriter('logs/test',sess.graph)
    for epoch in range(1001):
        for batch in range(n_batch):
            #使用50%的神经元用训练集对网络进行训练
            sess.run(train_step,feed_dict={x:X_train[batch*100:batch*100+99,:],y:y_train[batch*100:batch*100+99,:],keep_prob:0.5})
            #记录训练集计算的参数,使用训练集对网络进行测试
            summary = sess.run(merged,feed_dict={x:X_train[batch*100:batch*100+99,:],y:y_train[batch*100:batch*100+99,:],keep_prob:1.0})
            train_writer.add_summary(summary,epoch)
            #记录测试集计算的参数，使用测试集对网络进行测试
        summary = sess.run(merged,feed_dict={x:X_test,y:y_test,keep_prob:1.0})
        test_writer.add_summary(summary,epoch)
        
        if epoch%100==0:
            testing_acc = sess.run(accuracy,feed_dict={x:X_test,y:y_test,keep_prob:1.0})
            training_acc = sess.run(accuracy,feed_dict={x:X_train,y:y_train,keep_prob:1.0})
            print("iter"+str(epoch)+",testing accuracy"+str(testing_acc)+",traning accuracy"+str(training_acc))

TypeError: weight_variable() got an unexpected keyword argument 'name'